#### Script para recortar a base de Amazônia Legal por Estado e por Municípios

Projeto: Sistema de Apoio à Caracterização de Imóveis Rurais  
Embrapa/2023

In [1]:
# Módulos necessários
import geopandas as gpd
import pandas as pd
import os
import glob
import numpy as np

In [2]:
# Definir diretório principal
dirpath = input('Diretório principal: ')

In [3]:
# Definir caminho do shapefile arquivo original de Amazônia Legal
amz = input('Caminho Amazônia Legal: ')

In [4]:
# Definir caminho do shapefile que será usado como máscara para recorte (Municípios do BR - sem Buffer)
limites = input('Caminho Municípios: ')

In [6]:
# Ler o arquivo original de Amazônia Legal
base = gpd.read_file(amz)
base = base.to_crs(epsg=4326)
# Lendo shapefile de Municípios
lim = gpd.read_file(limites)
lim = lim.to_crs(epsg=4326)
# Criar pasta para armazenar a base de Amazônia Legal recortada por estado
uf_path = os.path.join(dirpath, 'Amz_Legal_Estados')
os.makedirs(uf_path, exist_ok=True)
# Criar pasta para armazenar a base de Amazônia Legal recortada por municípios
mun_path = os.path.join(dirpath, 'Amz_Legal_Municipios')
os.makedirs(mun_path, exist_ok=True)
# Cortar por estado
# Agrupar por estado
agrupado = lim.groupby('SIGLA_UF')
for key,values in agrupado:
    output = uf_path + f'\Amz_Legal_2021_{key}.shp'
    uf = lim[lim["SIGLA_UF"] == f"{key}"]
    geodf_clip = gpd.clip(base, uf, keep_geom_type=True)
    geodf_clip.to_file(driver = 'ESRI Shapefile', filename = (rf'{output}'))
# Lista dos estados
estados = list(np.unique(lim['SIGLA_UF']))
# Cortar por município
for estado in estados:
    shapes = glob.glob(uf_path + f'**/*{estado}.shp')
    # Selecionar GeoDataFrame por estado
    select = lim[lim['SIGLA_UF'] == f'{estado}']
    # Agrupar por município
    agrupado = select.groupby('CD_MUN')
    for shape in shapes:
        for key,values in agrupado:
            amz = gpd.read_file(shape)
            amz = amz.to_crs(epsg=4326)
            nome_arq = os.path.basename(shape)
            nome_arq = nome_arq.replace('.shp', f'_{key}.shp')
            output = mun_path + f"\{nome_arq}"
            mun = lim[lim["CD_MUN"] == f"{key}"]
            geodf_clip = gpd.clip(amz, mun, keep_geom_type=True)
            geodf_clip.to_file(driver = 'ESRI Shapefile', filename = (rf'{output}'))